**Importing Necessary Libraries**

In [1]:
import pandas as pd
import os
import re
import requests
from bs4 import BeautifulSoup
import string

In [2]:
#importing nltk package for working with NLP

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Connecting to the Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Data Extraction and NLP/"

**Importing Input Data**

In [5]:
input_df = pd.read_excel(f"{path}Input.xlsx")
input_df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


**Positive & Negative Words Dictionary**

In [6]:
PSF = f"{path}Dictionary/positive-words.txt"

with open(PSF, 'r') as pos:
  # Read the contents of the file and split them into a list of positive words using '\n' as the delimiter.
  positivewords = pos.read().split('\n')

NSF = f"{path}Dictionary/negative-words.txt"

with open(NSF, 'r', encoding="ISO-8859-1") as neg:
  # Read the contents of the file and split them into a list of negative words using '\n' as the delimiter.
  negativewords = neg.read().split('\n')

**StopWords Dictionary**

In [7]:
stopwords_files = f"{path}StopWords/"
stopwords_list = []

# Loop through files in the StopWords directory.
for filename in os.listdir(stopwords_files):
    if filename.endswith('.txt'):
      with open(os.path.join(stopwords_files, filename), 'r', encoding='ISO-8859-1') as file:
        for line in file:
          # Split the line using '|' as a delimiter and extract the first word, for currencies stopwords.
          words = line.split('|')[0].strip()
          stopwords_list.append(words)

stopwords_list = list(set(stopwords_list))   # Remove duplicate stopwords by converting the list to a set and back to a list.
stopwords_list.sort()                        # Sort the stopwords list alphabetically.

**Program to fetch links and perform required operations**

In [8]:
# Create an empty list to store the result data for each article.
result_data = []

# Loop through each row in the input dataset
for index, row in input_df.iterrows():
  article_url = row['URL']

  # Send an HTTP GET request to the article URL and parse the HTML content.
  response = requests.get(article_url)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Extract the article title from the HTML.
  article_title = soup.find('h1')
  if article_title:
    article_title_text = article_title.text
  else:
    article_title_text = "Title not found"

  # Extract the article text content from the HTML.
  article_text_elements = soup.findAll(attrs= {'class': 'td-post-content'})
  if article_text_elements:
      article_text = article_text_elements[0].text.replace('\n', " ")
  else:
      article_text = "Content not found"

  # Remove punctuation from the article text.
  article_text1 = article_text.translate(str.maketrans('','', string.punctuation))

  # Tokenize the cleaned article text into words.
  text_tokens = word_tokenize(article_text1)

  # Remove stopwords from the tokenized text.
  nostopwords_tokens = [word for word in text_tokens if not word in stopwords_list]

  # Calculate the number of words in the text.
  words_count = len(nostopwords_tokens)

  # Calculate positive and negative scores using dictionary.
  Positive_Score = sum(1 for word in nostopwords_tokens if word in positivewords)
  Negative_Score = sum(1 for word in nostopwords_tokens if word in negativewords)

  # Calculate Polarity and Subjectivity Scores.
  Polarity_Score = (Positive_Score - Negative_Score) / ((Positive_Score + Negative_Score) + 0.000001)
  Subjectivity_Score = (Positive_Score + Negative_Score) / ((words_count) + 0.000001)

  # Define functions to calculate average sentence length.
  def cal_avg_sentence_length(text):
    total_characters = len(re.sub(r'\s', '', text))
    total_sentences = len(re.split(r'[?!.]', text))
    avg_sentence_length = total_characters / total_sentences if total_sentences > 0 else 0
    return avg_sentence_length
  Average_Sentence_Length = cal_avg_sentence_length(article_text)

  # Define functions to calculate average number of words per sentence.
  def cal_avg_number_of_words_per_sentence(text):
    sentences = re.split(r'[.!?]', text)
    avg_number_of_words_per_sentence = sum(len(sentence.split()) for sentence in sentences) / len(sentences) if sentences else 0
    return avg_number_of_words_per_sentence
  Average_Number_Of_Words_Per_Sentence = cal_avg_number_of_words_per_sentence(article_text)

  # Define a function to count syllables in a word.
  def count_syllables(word):
    vowels = "AEIOUaeiou"
    syllable_count = 0
    prev_char_was_vowel = False
    for char in word:
        if char in vowels:
            if not prev_char_was_vowel:
                syllable_count += 1
            prev_char_was_vowel = True
        else:
            prev_char_was_vowel = False
    if word.endswith(('e', 'E')) and not word.endswith(('ed', 'es')):    # Adjustment for words ending with "ed" and "es"
        syllable_count -= 1
    syllable_count = max(1, syllable_count)
    return syllable_count
  words = re.findall(r'\b\w+\b', article_text)

  # Count the number of complex words based on syllable count.
  Complex_Word_Count = sum(1 for word in words if count_syllables(word) > 2)

  # Calculate the average syllables per word based on syllable count.
  syllable_counts = [count_syllables(word) for word in words]
  Syllable_Per_Word = sum(syllable_counts)/len(syllable_counts)

  # Calculate the word count after removing stopwords using stopwords class of nltk package.
  stop_words = stopwords.words('english')
  word_count = [word for word in text_tokens if not word in stop_words]
  Word_Count = len(word_count)

  # Calculate the percentage of complex words.
  Percentage_of_Complex_Words = (Complex_Word_Count / Word_Count) * 100

  # Calculate the Fog Index.
  Fog_Index = 0.4 * (Average_Sentence_Length + Percentage_of_Complex_Words)

  # Count personal pronouns in the article text.
  pronouns = r'\b(?:I|we|my|ours|us)\b'
  pronoun_matches = re.findall(pronouns, article_text, flags=re.IGNORECASE)
  filtered_pronouns = [pronoun for pronoun in pronoun_matches if pronoun.lower() != "us"]
  Personal_Pronouns = len(filtered_pronouns)

  # Calculate the average word length.
  total_characters = sum(len(word) for word in words)
  total_words = len(article_text.split())
  Average_Word_Length = total_characters / total_words if total_words > 0 else 0


  # Extract URL_ID and URL from the current row and append the result data to the list.
  url_id = row['URL_ID']
  url = row['URL']
  result_data.append({
      'URL_ID': url_id,
      'URL' : url,
      'POSITIVE SCORE': Positive_Score,
      'NEGATIVE SCORE': Negative_Score,
      'POLARITY SCORE': Polarity_Score,
      'SUBJECTIVITY SCORE': Subjectivity_Score,
      'AVG SENTENCE LENGTH': Average_Sentence_Length,
      'PERCENTAGE OF COMPLEX WORDS': Percentage_of_Complex_Words,
      'FOG INDEX': Fog_Index,
      'AVG NUMBER OF WORDS PER SENTENCE': Average_Number_Of_Words_Per_Sentence,
      'COMPLEX WORD COUNT': Complex_Word_Count,
      'WORD COUNT': Word_Count,
      'SYLLABLE PER WORD': Syllable_Per_Word,
      'PERSONAL PRONOUNS': Personal_Pronouns,
      'AVG WORD LENGTH': Average_Word_Length
  })

# Create a DataFrame from the result data.
results_df = pd.DataFrame(result_data)

**Rounding the values in numeric column to three decimal places**

In [9]:
for column in results_df.columns:
    if pd.api.types.is_numeric_dtype(results_df[column]):
        results_df[column] = results_df[column].round(3)

In [10]:
results_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,76,24,0.520,0.109,109.322,33.905,57.291,19.195,356,1050,1.801,2,5.603
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,38,13,0.490,0.165,135.960,40.563,70.609,24.480,144,355,1.736,3,5.448
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,19,27,-0.174,0.076,79.230,30.568,43.919,14.676,210,687,1.664,3,5.275
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,33,27,0.100,0.085,110.206,31.970,56.871,19.603,258,807,1.722,7,5.500
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,33,27,0.100,0.085,110.206,31.970,56.871,19.603,258,807,1.722,7,5.500


**Saving the Result dataset**

In [13]:
results_df.to_csv('Output Data.xlsx')